In [29]:
!pip install geopandas

In [30]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json

def setEdj(df: pd.DataFrame, key) -> None :
    df.loc[df[key].isin(['5', '6', '7']), key] = 'VET'
    df.loc[df[key].isin(['8', '9']), key] = 'GE'
    df.loc[df[key].isin(['2', '3', '4']), key] = 'IS'
    df.loc[df[key].isin(['1']), key] = 'NET'

In [31]:
merged_df = pd.read_csv("./study-data/TREE2_Data_IWI_Open_Data_Vorlesung_2023_nolabel.csv")

merged_df = merged_df[merged_df['t1bqvalids'] != 'No valid response']
merged_df = merged_df[merged_df['t2bqvalids'] != 'No valid response']
merged_df = merged_df[merged_df['t3bqvalids'] != 'No valid response']

merged_df.dropna(inplace=True)
merged_df['t1educ_class_1_r'] = merged_df['t1educ_class_1_r'].astype(int).astype(str)
merged_df['t2educ_class_1_r'] = merged_df['t2educ_class_1_r'].astype(int).astype(str)
merged_df['t3educ_class_1_r'] = merged_df['t3educ_class_1_r'].astype(int).astype(str)


setEdj(merged_df, 't1educ_class_1_r')
setEdj(merged_df, 't2educ_class_1_r')
setEdj(merged_df, 't3educ_class_1_r')

cols = ['aes_canton', 'aes_langreg', 't1wt', 't2wt', 't3wt', 't1educ_class_1_r', 't2educ_class_1_r', 't3educ_class_1_r']

merged_df = merged_df[cols]
merged_df.fillna(0, inplace=True)

merged_df['aes_langreg'] = merged_df['aes_langreg'].replace(1, 'GR')
merged_df['aes_langreg'] = merged_df['aes_langreg'].replace(2, 'FR')
merged_df['aes_langreg'] = merged_df['aes_langreg'].replace(3, 'IT')

merged_df.head(20)


,aes_canton,aes_langreg,t1wt,t2wt,t3wt,t1educ_class_1_r,t2educ_class_1_r,t3educ_class_1_r
1,10,FR,4.959451,5.249187,4.995442,IS,VET,VET
3,11,GR,9.184840,9.340135,13.201428,VET,VET,NET
6,15,GR,0.612540,0.618889,0.562086,VET,VET,VET
7,10,GR,10.286020,14.506459,10.028031,VET,VET,VET
8,10,GR,2.655471,2.107327,2.090831,GE,GE,GE
10,25,FR,3.366786,3.206607,3.349368,GE,GE,GE
11,15,GR,0.662122,0.778710,0.708539,IS,VET,VET
12,25,FR,11.310630,9.486566,35.055328,IS,NET,NET
13,1,GR,17.446143,15.878919,16.129011,VET,VET,VET
15,17,GR,3.042590,2.854162,2.945887,GE,GE,GE


In [32]:
# Grouping data by canton and education type and calculating the weighted count
w1_cantons = merged_df.groupby(['aes_canton', 't1educ_class_1_r']).apply(
    lambda x: (x['t1wt']).sum()).reset_index(name='w1_edu')

w2_cantons = merged_df.groupby(['aes_canton', 't2educ_class_1_r']).apply(
    lambda x: (x['t1wt']).sum()).reset_index(name='w2_edu')

w3_cantons = merged_df.groupby(['aes_canton', 't3educ_class_1_r']).apply(
    lambda x: (x['t1wt']).sum()).reset_index(name='w3_edu')


w1_cantons = w1_cantons.rename(columns={'t1educ_class_1_r': 'education'})
w2_cantons = w2_cantons.rename(columns={'t2educ_class_1_r': 'education'})
w3_cantons = w3_cantons.rename(columns={'t3educ_class_1_r': 'education'})


edu_count = pd.merge(w1_cantons, w2_cantons, on=['aes_canton', 'education'], how='outer')
edu_count = pd.merge(edu_count, w3_cantons, on=['aes_canton', 'education'], how='outer')

#merged_df.set_index('education', inplace=True)
edu_count.head(20)

C:\Users\CptMeh\AppData\Local\Temp\ipykernel_6284\2770027761.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  w1_cantons = merged_df.groupby(['aes_canton', 't1educ_class_1_r']).apply(
C:\Users\CptMeh\AppData\Local\Temp\ipykernel_6284\2770027761.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  w2_cantons = merged_df.groupby(['aes_canton', 't2educ_class_1_r']).apply(
C:\Users\CptMeh\AppData\Local\Temp\i

,aes_canton,education,w1_edu,w2_edu,w3_edu
0,1,GE,2822.459832,2875.675016,2792.182829
1,1,IS,1717.171951,330.527861,114.166294
2,1,NET,340.946749,165.893318,493.385885
3,1,VET,4949.327069,6457.809406,6430.170593
4,2,GE,2234.584649,2285.406745,2264.229984
5,2,IS,1346.252183,183.111332,81.462125
6,2,NET,123.627758,106.999650,208.084862
7,2,VET,3442.092653,4571.039517,4592.780272
8,3,GE,734.225355,786.724219,761.738568
9,3,IS,313.620559,87.777121,24.908537


In [33]:
# TODO: df.to_html('./test.html')

In [34]:
'''
Summarise the columns w1_weighted_count, w2_weighted_count, and w3_weighted_count across all aes_canton for each education category, 
and then append these summaries back to your original DataFrame.
'''

summary_df = edu_count.groupby('education').sum().reset_index()
summary_df.drop('aes_canton', axis=1, inplace=True)
summary_df['aes_canton'] = 'summary'
final_df = pd.concat([edu_count, summary_df], ignore_index=True)
final_df.fillna(0.0, inplace=True)

group_sums = final_df.groupby('aes_canton')[['w1_edu', 'w2_edu', 'w3_edu']].transform('sum')

# Convert each column to a percentage of its total
final_df['w1_edu'] = ((final_df['w1_edu'] / group_sums['w1_edu']) * 100).round(1)
final_df['w2_edu'] = ((final_df['w2_edu'] / group_sums['w2_edu']) * 100).round(1)
final_df['w3_edu'] = ((final_df['w3_edu'] / group_sums['w3_edu']) * 100).round(1)


final_df.to_csv('./study-data/currated_data.csv')
final_df.head(20)

,aes_canton,education,w1_edu,w2_edu,w3_edu
0,1,GE,28.7,29.3,28.4
1,1,IS,17.5,3.4,1.2
2,1,NET,3.5,1.7,5.0
3,1,VET,50.3,65.7,65.4
4,2,GE,31.3,32.0,31.7
5,2,IS,18.8,2.6,1.1
6,2,NET,1.7,1.5,2.9
7,2,VET,48.2,64.0,64.3
8,3,GE,26.3,28.1,27.2
9,3,IS,11.2,3.1,0.9


In [35]:
multi_index_df = final_df.set_index(['aes_canton', 'education'])
multi_index_df.to_csv('./study-data/currated_data.csv')
multi_index_df.loc[(4, 'IS')]

w1_edu    4.3
w2_edu    0.0
w3_edu    2.8
Name: (4, IS), dtype: float64

In [36]:
details = {}
for canton_id in multi_index_df.index.get_level_values('aes_canton').unique():
    canton_details = {}
    for edu_level in multi_index_df.xs(canton_id, level='aes_canton').index:
        canton_details[edu_level] = {
            'w1_edu': multi_index_df.loc[(canton_id, edu_level), 'w1_edu'],
            'w2_edu': multi_index_df.loc[(canton_id, edu_level), 'w2_edu'],
            'w3_edu': multi_index_df.loc[(canton_id, edu_level), 'w3_edu']
        }
    details[str(canton_id)] = canton_details


details

{'1': {'GE': {'w1_edu': 28.7, 'w2_edu': 29.3, 'w3_edu': 28.4},
  'IS': {'w1_edu': 17.5, 'w2_edu': 3.4, 'w3_edu': 1.2},
  'NET': {'w1_edu': 3.5, 'w2_edu': 1.7, 'w3_edu': 5.0},
  'VET': {'w1_edu': 50.3, 'w2_edu': 65.7, 'w3_edu': 65.4}},
 '2': {'GE': {'w1_edu': 31.3, 'w2_edu': 32.0, 'w3_edu': 31.7},
  'IS': {'w1_edu': 18.8, 'w2_edu': 2.6, 'w3_edu': 1.1},
  'NET': {'w1_edu': 1.7, 'w2_edu': 1.5, 'w3_edu': 2.9},
  'VET': {'w1_edu': 48.2, 'w2_edu': 64.0, 'w3_edu': 64.3}},
 '3': {'GE': {'w1_edu': 26.3, 'w2_edu': 28.1, 'w3_edu': 27.2},
  'IS': {'w1_edu': 11.2, 'w2_edu': 3.1, 'w3_edu': 0.9},
  'NET': {'w1_edu': 4.5, 'w2_edu': 2.7, 'w3_edu': 8.4},
  'VET': {'w1_edu': 58.0, 'w2_edu': 66.0, 'w3_edu': 63.4}},
 '4': {'GE': {'w1_edu': 18.5, 'w2_edu': 18.5, 'w3_edu': 17.5},
  'IS': {'w1_edu': 4.3, 'w2_edu': 0.0, 'w3_edu': 2.8},
  'NET': {'w1_edu': 0.0, 'w2_edu': 0.0, 'w3_edu': 0.6},
  'VET': {'w1_edu': 77.2, 'w2_edu': 81.5, 'w3_edu': 79.1}},
 '5': {'GE': {'w1_edu': 25.2, 'w2_edu': 21.9, 'w3_edu': 21.6}

In [37]:
sorted_by_wave = {}

# Iterate through each entry in the original dictionary
for key, categories in details.items():
    # Initialize the entry for each wave within this key
    sorted_by_wave[key] = {'w1_edu': {}, 'w2_edu': {}, 'w3_edu': {}}

    # Now, populate the new structure with data
    for category, waves in categories.items():
        for wave, count in waves.items():
            sorted_by_wave[key][wave][category] = count

# Now sorted_by_wave is your dictionary sorted by wave
details = sorted_by_wave


In [38]:
total = len(merged_df['aes_langreg'])

details['summary']['GR'] = (merged_df[merged_df['aes_langreg'] == 'GR'].count()[0] / total * 100).round(1)
details['summary']['FR'] = (merged_df[merged_df['aes_langreg'] == 'FR'].count()[0] / total * 100).round(1)
details['summary']['IT'] = (merged_df[merged_df['aes_langreg'] == 'IT'].count()[0] / total * 100).round(1)
details['summary']


C:\Users\CptMeh\AppData\Local\Temp\ipykernel_6284\3096745451.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  details['summary']['GR'] = (merged_df[merged_df['aes_langreg'] == 'GR'].count()[0] / total * 100).round(1)
C:\Users\CptMeh\AppData\Local\Temp\ipykernel_6284\3096745451.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  details['summary']['FR'] = (merged_df[merged_df['aes_langreg'] == 'FR'].count()[0] / total * 100).round(1)
C:\Users\CptMeh\AppData\Local\Temp\ipykernel_6284\3096745451.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will al

{'w1_edu': {'GE': 34.8, 'IS': 14.2, 'NET': 4.0, 'VET': 47.0},
 'w2_edu': {'GE': 35.9, 'IS': 3.2, 'NET': 2.5, 'VET': 58.4},
 'w3_edu': {'GE': 34.8, 'IS': 1.5, 'NET': 4.6, 'VET': 59.1},
 'GR': 69.2,
 'FR': 25.3,
 'IT': 5.5}

In [39]:
details['summary']['GR']

69.2

In [40]:

tot_lang = details['summary']['GR'] + details['summary']['FR'] + details['summary']['IT']

summary = { 'w1_edu' : {'GE' : details['summary']['w1_edu']['GE'],
                        'VET' : details['summary']['w1_edu']['VET'],
                        'IS' : details['summary']['w1_edu']['IS'],
                        'NET' : details['summary']['w1_edu']['NET']},

            'w2_edu' : {'GE' : details['summary']['w2_edu']['GE'],
                        'VET' : details['summary']['w2_edu']['VET'],
                        'IS' : details['summary']['w2_edu']['IS'],
                        'NET' : details['summary']['w2_edu']['NET']},

            'w3_edu' : {'GE' : details['summary']['w3_edu']['GE'],
                        'VET' : details['summary']['w3_edu']['VET'],
                        'IS' : details['summary']['w3_edu']['IS'],
                        'NET' : details['summary']['w3_edu']['NET']},

            'lang': {'GR' : details['summary']['GR'],
                    'FR' : details['summary']['FR'],     
                    'IT' : details['summary']['IT']}}



with open('./study-data/summary.json', 'w') as file:
    json.dump(summary, file, indent=4)

In [41]:
!pip install geopandas

In [42]:
swiss_map = gpd.read_file('./geo-data/landesforstinventar-kantone_2056.geojson')

# Ensure the KantonId in GeoDataFrame is treated as a string if your details keys are strings
swiss_map['KantonId'] = swiss_map['KantonId'].astype(str)

# Merge the details into the GeoDataFrame
# This step adds a new column 'details' to the GeoDataFrame
swiss_map['details'] = swiss_map['KantonId'].map(details)

# Check the resulting GeoDataFrame
swiss_map.to_file('./geo-data/map_data.geojson', driver="GeoJSON")

swiss_map.head(5)

,KantonId,GeometrieId,KantonName_de,KantonName_fr,KantonName_it,KantonName_en,alternateName,geometry,details
0,1,1,Zürich,Zurich,Zurigo,Zurich,ZH,"POLYGON ((2692443.001 1281183.001, 2692994.001...","{'w1_edu': {'GE': 28.7, 'IS': 17.5, 'NET': 3.5..."
1,2,2,Bern,Berne,Berna,Bern,BE,"MULTIPOLYGON (((2573533.001 1194765.001, 25740...","{'w1_edu': {'GE': 31.3, 'IS': 18.8, 'NET': 1.7..."
2,3,3,Luzern,Lucerne,Lucerna,Lucerne,LU,"POLYGON ((2662029.001 1237691.001, 2662264.001...","{'w1_edu': {'GE': 26.3, 'IS': 11.2, 'NET': 4.5..."
3,4,4,Uri,Uri,Uri,Uri,UR,"POLYGON ((2688457.001 1203340.001, 2688740.001...","{'w1_edu': {'GE': 18.5, 'IS': 4.3, 'NET': 0.0,..."
4,5,5,Schwyz,Schwytz,Svitto,Schwyz,SZ,"POLYGON ((2704723.001 1229925.001, 2706210.001...","{'w1_edu': {'GE': 25.2, 'IS': 7.4, 'NET': 0.9,..."
